## Huggingface Datasets

In [2]:
# FROM 2012 to 2023, IMAGE Processing is Lagging behind NLP because of Image Size, Volume etc
# NLP because it's smaller, it potentially is a easier thing to practically do. But until 2019, even google didn't follow through

# ! KEY CODE
import datasets as huggingface_datasets
import torchvision

dataset = huggingface_datasets.load_dataset("zh-plus/tiny-imagenet")
print(huggingface_datasets.get_dataset_split_names("zh-plus/tiny-imagenet"))
training_dataset, validation_dataset = dataset['train'], dataset['valid']


transformations_list = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
    ])

def transform_datasets(examples):
    examples["converted_tensors"] = []
    
    for image in examples['image']:
        transformed_image = transformations_list(image)
        examples['converted_tensors'].append(transformed_image)
    
    return examples
 
training_dataset.set_transform(transform_datasets)
validation_dataset.set_transform(transform_datasets)

['train', 'valid']


In [5]:
training_dataset[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=64x64>,
 'label': 0,
 'converted_tensors': tensor([[[1.0000, 1.0000, 0.9647,  ..., 0.8157, 0.8118, 0.4902],
          [1.0000, 1.0000, 0.9647,  ..., 0.7961, 0.7843, 0.4745],
          [1.0000, 1.0000, 1.0000,  ..., 0.8118, 0.7843, 0.4667],
          ...,
          [0.3686, 0.3647, 0.3647,  ..., 0.3333, 0.3059, 0.3686],
          [0.3490, 0.3451, 0.3412,  ..., 0.3373, 0.3020, 0.3412],
          [0.3569, 0.3569, 0.3569,  ..., 0.3765, 0.3020, 0.2824]],
 
         [[0.5373, 0.5451, 0.5804,  ..., 0.9294, 0.9451, 0.6431],
          [0.4863, 0.5020, 0.5373,  ..., 0.9098, 0.9216, 0.6275],
          [0.4863, 0.4980, 0.5255,  ..., 0.9137, 0.9216, 0.6118],
          ...,
          [0.4784, 0.4667, 0.4549,  ..., 0.2902, 0.2627, 0.3255],
          [0.4627, 0.4549, 0.4431,  ..., 0.2941, 0.2588, 0.2980],
          [0.4706, 0.4667, 0.4588,  ..., 0.3333, 0.2588, 0.2392]],
 
         [[0.7529, 0.7529, 0.7725,  ..., 0.9216, 0.9412, 0.6314],

## Model Architecture

In [4]:
import torch, torch.nn as nn
from torchinfo import summary

simple_model = nn.Sequential(
    nn.Conv2d(out_channels = 50001, in_channels = 3, kernel_size = (64, 64) ),
    nn.ReLU(),

    nn.Flatten(start_dim=1),
    
    nn.Linear(out_features = 10, in_features = 50)
)

from torchinfo import summary
summary(simple_model, input_size=(1,3,64,64), 
        verbose=2, col_names = ["input_size", "output_size","kernel_size", "num_params","trainable", "params_percent"]);

Layer (type:depth-idx)                   Input Shape               Output Shape              Kernel Shape              Param #                   Trainable                 Param %
Sequential                               [1, 3, 64, 64]            [1, 10]                   --                        --                        True                           --
├─Conv2d: 1-1                            [1, 3, 64, 64]            [1, 50, 1, 1]             [64, 64]                  614,450                   True                       99.92%
│    └─weight                                                                                [3, 50, 64, 64]           ├─614,400
│    └─bias                                                                                  [50]                      └─50
├─ReLU: 1-2                              [1, 50, 1, 1]             [1, 50, 1, 1]             --                        --                        --                             --
├─Flatten: 1-3                 

/Users/ajinkya/opt/anaconda3/lib/python3.9/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),


In [ ]:
import torch, torch.nn as nn

class Custom_Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Conv2d(out_channels = 5, in_channels = 3, kernel_size = (64, 64) )
        self.layer2 = nn.Linear(out_features = 8, in_features = 5)

    def forward(self, input_batch):
        layer1_output = self.layer1(input_batch)
        # layer1_output = torch.dot(input_batch, self.layer1.weight)

        activation_function_output = nn.functional.relu(layer1_output)

        layer2_output = self.layer2(activation_function_output)
        activation_function_output = nn.functional.relu(layer2_output)

        return activation_function_output

model = Custom_Model()

single_random_batch = torch.randn(1,3,64,64)
model(single_random_batch)

![](https://www.ophthalmologytraining.com/images/Stills/Visual%20Cortex.jpg)
![](https://substackcdn.com/image/fetch/w_1456,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F9b02ffb4-e1c2-46ba-9170-397bd0ececae_1280x451.png)

In [ ]:
import torch, torch.nn as nn
# Data: CAT OR A DOG DETECTOR
# Data: MICRO FRACTURE DETECTOR FROM XRAY
# Data: Satellite Data. We want to extract population & farming stats. 
# TODO: MODEL 
feature_extractor = nn.Sequential(
    nn.Conv2d( out_channels = 10, in_channels = 3, kernel_size = (3,3)), # visual cortex, v1: Lines & Edges
    nn.ReLU(),
  
    nn.Conv2d(out_channels = 10, in_channels = 10, kernel_size = (3,3)), # visual cortex, v2: Textures & Patterns
    nn.ReLU(),

    nn.Conv2d(out_channels = 10, in_channels = 10, kernel_size = (3,3)), 
    # visual cortex, v3: Parts of Objects. (Extracting all parts of object for all classes)
    nn.ReLU(),

    nn.Conv2d(out_channels = 2, in_channels = 10, kernel_size = (3,3)), # visual cortex, v4: Objects
    # all parts belonging to the same object, are combined and we have a single neuron, which detects, that specfic object. 
    nn.ReLU(),
)

decision_maker = nn.Sequential(
  nn.Linear(out_features = 50, in_features = 100*7*7 ),
  nn.Linear(out_features = 10, in_features = 50)
)

model = nn.Sequential(
  feature_extractor,
  decision_maker
)

In [10]:

x.shape

torch.Size([3, 64, 64])

In [ ]:
feature_extractor = nn.Sequential(
    nn.Conv2d( out_channels = 10, in_channels = 3, kernel_size = (3,3)), # visual cortex, v1: Lines & Edges
    nn.ReLU(),
  
    nn.Conv2d(out_channels = 10, in_channels = 10, kernel_size = (3,3)), # visual cortex, v2: Textures & Patterns
    nn.ReLU(),

    nn.Conv2d(out_channels = 10, in_channels = 10, kernel_size = (3,3)), 
    # visual cortex, v3: Parts of Objects. (Extracting all parts of object for all classes)
    nn.ReLU(),

    nn.Conv2d(out_channels = 2, in_channels = 10, kernel_size = (3,3)), # visual cortex, v4: Objects 
    # all parts belonging to the same object, are combined and we have a single neuron, which detects, that specfic object. 
    nn.ReLU(),
)



In [1]:
import torch, torch.nn as nn
class BASELINE_MODEL(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.v1 = nn.Conv2d( out_channels = 10, in_channels = 3, kernel_size = (4,4)) # visual cortex, v1: Lines & Edges

    # FORWARD. DATA BATCH, is passed left to right, through the layers, one by one. 
    def forward(self, input_batch):
        v1_output = self.v1(input_batch)
        # v1_ouput -> feature_map
        shape = v1_output.shape
        v1_activation_output = nn.functional.relu(v1_output)

x = torch.randn(1,3,6,6)

model = BASELINE_MODEL()
model(x)


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/Users/ajinkya/opt/anaconda3/lib/python3.9/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/Users/ajinkya/opt/anaconda3/lib/python3.9/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
x = torch.randn(1,3,16,16)
layer = nn.Conv2d( out_channels = 10, in_channels = 3, kernel_size = (9,9), stride = 4) # kernel = (3,8,8) image = 3,16,16
# output = 10, 2, 2

layer(x).shape

l2 = nn.Conv2d( out_channels = 20, in_channels = 10, kernel_size = (8,8))
l2(layer(x)).shape

torch.Size([1, 20, 2, 2])

In [ ]:
# DRIVER, MECHANIC, AUTOMOBILE ENGINEER
# ! MATHS GRADIENT DESCENT, MATHS LINEAR ALGEBRA